In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import json
import random

In [2]:
HEX_OPACITY = 0.3

with open('data/california_counties.json') as f:
  cali_geojson = json.load(f)

df = pd.DataFrame([
  [
    feature['properties']['NAME'], 
    random.random()
  ] for feature in cali_geojson['features']
], columns=['NAME', 'FIRE_PROBABILITY'])

car_df = px.data.carshare()

locations = json.load(open('data/locations.json'))
mock_df = pd.DataFrame(
  [location + [random.random()] for location in locations],
  columns=['LAT', 'LON', 'FIRE_SCORE']
)

In [4]:
ca_bounds_lat_lon = {
  'lon_range': [-124.409591, -114.131211],
  'lat_range': [32.534156, 42.009518]
}

ca_bounds_dash_coords = {
  'x_range': [-2.1713569841888964, -1.9919654110589782],
  'y_range': [0.5964742403574204, 0.8139749637349287]
}

ca_bound_locations = [
  [42.009518, -124.409591],
  [32.534156, -114.131211]
]

def dash_coord_to_lat_lon_california(location):
  x, y = location
  x_range = ca_bounds_dash_coords['x_range']
  y_range = ca_bounds_dash_coords['y_range']
  lon_range = ca_bounds_lat_lon['lon_range']
  lat_range = ca_bounds_lat_lon['lat_range']
  
  x_std = (x - x_range[0]) / abs(x_range[1] - x_range[0])
  y_std = (y - y_range[0]) / abs(y_range[1] - y_range[0])

  x_proj = lon_range[0] + x_std * (lon_range[1] - lon_range[0])
  y_proj = lat_range[0] + y_std * (lat_range[1] - lat_range[0])

  return [x_proj, y_proj]

print(dash_coord_to_lat_lon_california([-1.9919654110589782, 0.5964742403574204]))

[-114.131211, 32.534156]


In [6]:
def y_to_lat(y):
  return (
    2 * 
    (
      np.arctan(
        np.exp(y) 
      ) -
      (np.pi / 4.0)
    )
  )

y_to_lat(0.8)

0.7262048227415292

In [ ]:
fire_prob_median_fig = ff.create_hexbin_mapbox(
    data_frame=mock_df, lat="LAT", lon="LON",
    mapbox_style='open-street-map',
    nx_hexagon=12, opacity=HEX_OPACITY, labels={"color": "Median fire chance"},
    color="FIRE_SCORE", agg_func=np.median,
    min_count=1, width=600, height=600
)

fire_prob_median_fig.show()

In [ ]:
fig = px.choropleth(df, geojson=cali_geojson,
                    locations="NAME", featureidkey="properties.NAME",
                    projection="mercator"
                   )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig.layout.geo